<a href="https://colab.research.google.com/github/NEGU93/keras-tcn/blob/master/af_challenge.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install tensorflow==2.12
!pip install keras-tcn

In [2]:
import numpy as np
import pandas as pd
from pathlib import Path
import plotly.express as px
from datetime import datetime, timedelta
from sklearn.preprocessing import StandardScaler
import tensorflow as tf
from tcn import TCN

# Data Exploration

In [3]:
df = pd.read_csv(Path('/content/lounge_attendance.csv'))
df.date = pd.to_datetime(df.date)
df

,date,lounge_name,total_guests
0,2017-01-01,EK,1295
1,2017-01-01,EL,1294
2,2017-01-01,EM,738
3,2017-01-01,F,869
4,2017-01-01,G,128
...,...,...,...
5442,2019-12-29,EK,1527
5443,2019-12-29,EL,1494
5444,2019-12-29,EM,1018
5445,2019-12-29,F,2042


In [4]:
df.dtypes

,0
date,datetime64[ns]
lounge_name,object
total_guests,int64


In [5]:
# I usually start plotting the data
px.line(df, x='date', y='total_guests', facet_col='lounge_name', facet_col_wrap=3)

In [6]:
fig = px.box(df, x='lounge_name', y='total_guests',  notched=True, color='lounge_name', points="all")
fig.update_traces(boxmean=True)
fig.show()
# Data seems to be unskewed (mean close to median)

In [7]:
# Note: Data is from 1st of Jan 2017 to almost end of December 2019. So per year I have all days and month, making stats easier to compute seasonaly.


# And getting some stats with the describe feature
df.groupby('lounge_name')['total_guests'].describe()

,count,mean,std,min,25%,50%,75%,max
lounge_name,,,,,,,,
EK,1093.0,1707.243367,273.147938,733.0,1524.0,1688.0,1883.00,2641.0
EL,1093.0,1432.578225,292.187065,638.0,1218.0,1406.0,1638.00,2291.0
EM,1092.0,1022.134615,203.687926,492.0,883.5,1013.0,1158.00,1737.0
F,1093.0,2493.444648,591.530569,558.0,2040.0,2523.0,2985.00,3919.0
G,1076.0,485.647770,144.527714,4.0,376.0,492.0,599.25,813.0


In [8]:
# It seems I have missing dates, let's found them
df.date.max() - df.date.min()

Timedelta('1092 days 00:00:00')

In [9]:
def find_missing_dates(df):
    """
    For a given df with columns :code:`lounge_name` and :code:`date`, it finds for every different lounge name, the dates missing, assuming a frequency of one day.
    :param df: Dataframe with columns columns :code:`lounge_name` (str) and :code:`date` (datetime).
    :retruns: dictionary with keys the lounge names and values the missing dates.
    """
    max_date = df.date.max()
    min_date = df.date.min()
    missing_dates_dict = {}

    for lounge_name, group in df.groupby('lounge_name'):
        date_range = pd.date_range(start=min_date, end=max_date)
        missing_dates = date_range.difference(group['date'])
        missing_dates_dict[lounge_name] = missing_dates.tolist()
    return missing_dates_dict

missing_dates = find_missing_dates(df)
missing_dates

{'EK': [],
 'EL': [],
 'EM': [Timestamp('2018-02-08 00:00:00')],
 'F': [],
 'G': [Timestamp('2018-02-08 00:00:00'),
  Timestamp('2019-07-01 00:00:00'),
  Timestamp('2019-07-02 00:00:00'),
  Timestamp('2019-07-03 00:00:00'),
  Timestamp('2019-07-04 00:00:00'),
  Timestamp('2019-07-05 00:00:00'),
  Timestamp('2019-07-06 00:00:00'),
  Timestamp('2019-07-07 00:00:00'),
  Timestamp('2019-07-08 00:00:00'),
  Timestamp('2019-07-09 00:00:00'),
  Timestamp('2019-07-10 00:00:00'),
  Timestamp('2019-07-11 00:00:00'),
  Timestamp('2019-07-12 00:00:00'),
  Timestamp('2019-07-13 00:00:00'),
  Timestamp('2019-07-14 00:00:00'),
  Timestamp('2019-07-15 00:00:00'),
  Timestamp('2019-07-16 00:00:00')]}

In [10]:
# I want to check week, month and year seasonality
df['day_of_week'] = df.date.dt.day_name()
df['month'] = df.date.dt.month
df['year'] = df.date.dt.year
df.head()

,date,lounge_name,total_guests,day_of_week,month,year
0,2017-01-01,EK,1295,Sunday,1,2017
1,2017-01-01,EL,1294,Sunday,1,2017
2,2017-01-01,EM,738,Sunday,1,2017
3,2017-01-01,F,869,Sunday,1,2017
4,2017-01-01,G,128,Sunday,1,2017


In [11]:
# There seems indeed to be a high difference between weekdays, but I would rather see the percentaje to have an idea
df.groupby(['lounge_name', 'day_of_week'])['total_guests'].describe()

count         mean         std     min      25%  \
lounge_name day_of_week                                                    
EK          Friday       156.0  1633.493590  206.420101  1069.0  1509.75   
            Monday       156.0  1952.596154  286.604882  1155.0  1782.75   
            Saturday     156.0  1708.288462  222.770300  1087.0  1590.75   
            Sunday       157.0  1879.891720  265.366186  1062.0  1749.00   
            Thursday     156.0  1539.551282  206.627428   781.0  1414.25   
            Tuesday      156.0  1627.275641  237.369769   733.0  1493.00   
            Wednesday    156.0  1608.500000  201.350521   927.0  1479.75   
EL          Friday       156.0  1337.230769  228.045011   759.0  1169.75   
            Monday       156.0  1713.442308  278.320730   779.0  1578.50   
            Saturday     156.0  1452.910256  252.810228   755.0  1256.75   
            Sunday       157.0  1647.280255  257.276137   803.0  1501.00   
            Thursday     156.0  1251.557692  221.086970   638.0  1086.75   
            Tuesday      156.0  1342.717949  233.378740   669.0  1199.25   
            Wednesday    156.0  1281.532051  195.972724   758.0  1152.25   
EM          Friday       156.0   980.089744  167.299283   578.0   865.25   
            Monday       156.0  1186.134615  175.846197   745.0  1067.25   
            Saturday     156.0  1069.493590  136.371717   712.0   979.50   
            Sunday       157.0  1194.343949  178.796465   684.0  1070.00   
            Thursday     155.0   880.025806  150.518970   492.0   773.00   
            Tuesday      156.0   936.500000  165.609393   556.0   818.75   
            Wednesday    156.0   906.339744  173.762259   537.0   788.00   
F           Friday       156.0  2877.826923  555.965248  1603.0  2525.25   
            Monday       156.0  2748.557692  615.374905  1112.0  2362.75   
            Saturday     156.0  1980.448718  362.556234   749.0  1782.50   
            Sunday       157.0  2055.000000  350.992549   869.0  1892.00   
            Thursday     156.0  2753.519231  508.363884  1489.0  2401.00   
            Tuesday      156.0  2465.365385  527.774195   558.0  2198.25   
            Wednesday    156.0  2576.205128  485.196903  1254.0  2309.25   
G           Friday       154.0   568.266234  132.038222   199.0   493.25   
            Monday       153.0   586.307190  158.635766   113.0   509.00   
            Saturday     154.0   351.701299   72.983939   143.0   305.25   
            Sunday       155.0   378.483871   82.173506   128.0   327.00   
            Thursday     153.0   516.464052  118.350340   142.0   449.00   
            Tuesday      153.0   497.660131  124.134433   112.0   451.00   
            Wednesday    154.0   502.279221  122.121203     4.0   455.00   

                            50%      75%     max  
lounge_name day_of_week                           
EK          Friday       1642.5  1763.50  2200.0  
            Monday       1948.5  2143.50  2641.0  
            Saturday     1699.5  1862.25  2305.0  
            Sunday       1903.0  2084.00  2545.0  
            Thursday     1529.0  1657.25  2122.0  
            Tuesday      1641.0  1799.25  2155.0  
            Wednesday    1620.0  1735.25  2148.0  
EL          Friday       1325.5  1490.25  2074.0  
            Monday       1741.5  1908.50  2219.0  
            Saturday     1431.5  1645.00  2097.0  
            Sunday       1662.0  1822.00  2259.0  
            Thursday     1222.0  1398.75  1910.0  
            Tuesday      1341.5  1498.75  2291.0  
            Wednesday    1262.5  1409.50  1752.0  
EM          Friday        984.0  1090.00  1540.0  
            Monday       1181.5  1324.25  1603.0  
            Saturday     1064.0  1163.00  1400.0  
            Sunday       1194.0  1324.00  1684.0  
            Thursday      896.0   974.00  1315.0  
            Tuesday       952.0  1050.50  1342.0  
            Wednesday     895.0  1008.00  1737.0  
F           Friday       3039.0  3304.5

In [12]:
# Instead of showing the pandas table percentages, let's do some plotly showoff
fig = px.sunburst(
    df,
    path=['lounge_name', 'day_of_week'],
    values='total_guests',
)
fig.update_traces(textinfo="label+percent parent")
fig.show()
# Conlcusions from the graph:
# - Monday has normally higher afluence (and high if not)
# - Thursday lower affluence, except for F which is quite high.

In [13]:
# Let's check monthly seasonality, with another graph to show I am resourceful.
px.histogram(df, x='month', color='lounge_name', y='total_guests', barmode='group')
# Conclusions from this graph: There are peaks around October and another around June.

In [14]:
px.histogram(df, x='year', color='lounge_name', y='total_guests', barmode='group')
# TIP: F lounge makes it hard to see the effect for G for example, but you can show only G with plotly so you will the tendence.
# Per year, there is a tendency to increase afluence, except for lounge G. The weeks on July the lounge was "closed" might have something to do with this. July is a high afluence month.

In [15]:
# Create lagged features for previous date, day of week, and month
df['previous_date_total_guests'] = df.groupby('lounge_name')['total_guests'].shift(1)
df['previous_day_of_week_total_guests'] = df.groupby(['lounge_name', 'day_of_week'])['total_guests'].shift(1)
df['previous_month_total_guests'] = df.groupby(['lounge_name', 'month'])['total_guests'].shift(1)

# Calculate correlations
correlation_previous_date = df['total_guests'].corr(df['previous_date_total_guests'])
correlation_previous_day_of_week = df['total_guests'].corr(df['previous_day_of_week_total_guests'])
correlation_previous_month = df['total_guests'].corr(df['previous_month_total_guests'])

print(f"Correlation with previous date's total guests: {correlation_previous_date}")
print(f"Correlation with previous day of week's total guests: {correlation_previous_day_of_week}")
print(f"Correlation with previous month's total guests: {correlation_previous_month}")

Correlation with previous date's total guests: 0.911761289759513
Correlation with previous day of week's total guests: 0.9482434342306539
Correlation with previous month's total guests: 0.9082786915821456


## Data exploration conclusions
- Data is from 1st of Jan 2017 to almost end of December 2019.
- Seems to be unskewed.
- I have missing dates for lounge EM (only one) and G (about a week, probably the lounge was closed).
  - Did this generated more affluence to other lounges?
- There seems to be a strong seasonality, per month and day of the week, as well as some tendency to increase per year.

# Pre-processing

Objectives:
- Deal with missing values
- Get train-test set

In [16]:
# Create a copy of the DataFrame to avoid modifying the original
df_filled = df.copy()

# Iterate through each lounge and its missing dates
for lounge, missing_dates_list in missing_dates.items():
    for missing_date in missing_dates_list:
        # Find the previous week's date
        previous_week_date = missing_date - pd.Timedelta(days=7)

        # Find the total_guests for the previous week's date for the current lounge
        previous_week_guests = df_filled[
            (df_filled['lounge_name'] == lounge) & (df_filled['date'] == previous_week_date)
        ]['total_guests']

        # If the previous week's data exists, fill the missing date with that value
        if not previous_week_guests.empty:
            new_row = pd.DataFrame({
                'lounge_name': [lounge],
                'date': [missing_date],
                'total_guests': [previous_week_guests.iloc[0]],
                'day_of_week': [missing_date.day_name()],
                'month': [missing_date.month],
                'year': [missing_date.year]
            })
            df_filled = pd.concat([df_filled, new_row], ignore_index=True)

# Sort the DataFrame by date
df_filled = df_filled.sort_values(by='date').reset_index(drop=True)

# Recalculate lagged features (optional, but recommended)
df_filled['previous_date_total_guests'] = df_filled.groupby('lounge_name')['total_guests'].shift(1)
df_filled['previous_day_of_week_total_guests'] = df_filled.groupby(['lounge_name', 'day_of_week'])['total_guests'].shift(1)
df_filled['previous_month_total_guests'] = df_filled.groupby(['lounge_name', 'month'])['total_guests'].shift(1)

In [17]:
# Verify my code worked :)
df_filled[
    (df_filled['lounge_name'] == 'G') &
    (df_filled['date'] > datetime(year=2019, month=6, day=23)) &
    (df_filled['date'] < datetime(year=2019, month=7, day=20))
  ]

,date,lounge_name,total_guests,day_of_week,month,year,previous_date_total_guests,previous_day_of_week_total_guests,previous_month_total_guests
4524,2019-06-24,G,761,Monday,6,2019,510.0,593.0,510.0
4525,2019-06-25,G,603,Tuesday,6,2019,761.0,498.0,761.0
4534,2019-06-26,G,608,Wednesday,6,2019,603.0,545.0,603.0
4535,2019-06-27,G,695,Thursday,6,2019,608.0,519.0,608.0
4543,2019-06-28,G,645,Friday,6,2019,695.0,579.0,695.0
4549,2019-06-29,G,390,Saturday,6,2019,645.0,446.0,645.0
4550,2019-06-30,G,418,Sunday,6,2019,390.0,510.0,390.0
4559,2019-07-01,G,761,Monday,7,2019,418.0,761.0,327.0
4560,2019-07-02,G,603,Tuesday,7,2019,761.0,603.0,761.0
4568,2019-07-03,G,608,Wednesday,7,2019,603.0,608.0,603.0


In [18]:
# Another verification
df_filled.groupby('lounge_name')['total_guests'].count()

,total_guests
lounge_name,
EK,1093
EL,1093
EM,1093
F,1093
G,1093


In [19]:
# Can't stop testing my code worked.
# Here I just verify all columns in df are also in df_filled... So that I kept all values.
for index, row in df.iterrows():
    # Find the corresponding row in df_filled
    matching_rows = df_filled[
        (df_filled['lounge_name'] == row['lounge_name']) &
        (df_filled['date'] == row['date'])
    ]
    # Check if a matching row exists
    if not matching_rows.empty:
        # Check if all values in the matching row are equal to the original row
        matching_row = matching_rows.iloc[0]
        if not all(row[col] == matching_row[col] for col in ['date', 'total_guests', 'lounge_name']):
            print(f"Mismatch found for row in df:\n{row}\nMatching row in df_filled:\n{matching_row}")
    else:
        print(f"Row in df not found in df_filled:\n{row}")

In [20]:
# I think using 2019-06-30 as train - test split will be Ok, therefore I prevent training on new generated data.
def train_test_split(df_filled, split_date: datetime):
    """
    Splits a dataframe into train and validation using a date as a split.
    It also scales the dataframes and returns the scaler used for that.

    .. warning::

      Data verification like the value of split_date is between the df dates are not done.

    :param df: Dataframe to be splitted with columns :code:`date` and :code:`total_guests`.
    :param split_date: Date to split the dataframe.
    :returns: numpy arrays of train and validation data. With shapes (number of lounges, timeseries of total guests, 1).

      - :code:`train_np`: numpy array with dates before :code:`split_date`
      - :code:`val_np`: numpy array with dates after :code:`split_date`
      - :code:`all_data_np`: numpy array with all the data. second dimension should be the sum of :code:`train_np` and :code:`val_np` second dimension.
      - :code:`scalers`: list of scalers used to scale the data.
    """
    train_np = []
    val_np = []
    all_data_np = []
    scalers = []
    for lounge in df_filled['lounge_name'].unique():
        lounge_df = df_filled[df_filled['lounge_name'] == lounge]
        scaler = StandardScaler()
        # Fit my scaler with all the data
        all_data_np.append(scaler.fit_transform(np.expand_dims(lounge_df['total_guests'].to_numpy(), axis=-1)))
        train_np.append(scaler.transform(np.expand_dims(lounge_df[lounge_df['date'] <= split_date]['total_guests'].to_numpy(), axis=-1)))
        val_np.append(scaler.transform(np.expand_dims(lounge_df[lounge_df['date'] > split_date]['total_guests'].to_numpy(), axis=-1)))
        scalers.append(scaler)   # I save the scaler to un-transform the predictions.

    val_np = np.array(val_np)
    all_data_np = np.array(all_data_np)
    train_np = np.array(train_np)
    return train_np, val_np, all_data_np, scalers


split_date=datetime(year=2019, month=6, day=30)
train_np, val_np, all_data_np, scalers = train_test_split(df_filled, split_date=split_date)
print(train_np.shape)
print(val_np.shape)
print(all_data_np.shape)

(5, 911, 1)
(5, 182, 1)
(5, 1093, 1)


In [21]:
scalers[0].mean_

array([1707.24336688])

# Train

## Model ideas

- I know SARIMA-base models deal well with seasonal and trend timeseries. It could be a strong method to substract information about this effect, and would also offer some data explicability (we could conclude the trend from year to year for excample).
- Meta [Prophet](https://facebook.github.io/prophet/). seems to be quite tailored for this task. Here their own description: *"Prophet is a procedure for forecasting time series data based on an additive model where non-linear trends are fit with yearly, weekly, and daily seasonality, plus holiday effects. It works best with time series that have strong seasonal effects and several seasons of historical data. Prophet is robust to missing data and shifts in the trend, and typically handles outliers well."*.
- Just some regression with Random Forest, GBoost, etc.
- Neural Networks: LSTMs or TCN algorithms (or Mamba / Transformers).

Prophet seems quite an interesting choice, but I have never tried it or used it, so I fear loosing too much time.
I decided to use TCN because it has some complexity (I allows me to showoff, using tf or Pytorch instead of sklearn), it might be a little overkill (but less overkill than Transformers).
In my experience, TCN works similarly in performance than LSTM but are quite easier to train (normally works without any parameter tuning).

I would like to first construct a benchmark to compare my result. I used a "Last day of the week" prediction because it has the higher correlation.

## Training method

I have 3 options for training the TCN:"

1. (BAD) Make model predict D-day + 14.
2. Iterative prediction (predict D-day+1 and use it iteratively to predict all 14 days).
3. Predict 14 days at once.

I choose option 3.

In [22]:
input_layer = tf.keras.Input(shape=(None, 1))
tcn_block = TCN(
    nb_filters=64,
    kernel_size=3,
    nb_stacks=1,
    dilations=(1, 2, 4, 8, 16, 32),
    padding='causal',
    use_skip_connections=True,
    dropout_rate=0.0,
    return_sequences=True,
    activation='relu',
    kernel_initializer='he_normal',
    use_batch_norm=False,
    use_layer_norm=False,
    use_weight_norm=False,
    go_backwards=False,
    return_state=False,
)(input_layer)
dense = tf.keras.layers.Dense(14)(tcn_block)
output_layer = tf.keras.layers.Activation('linear')(dense)
tcn_model = tf.keras.Model(inputs=input_layer, outputs=output_layer)

tcn_model.compile(optimizer='adam', loss='mse')
tcn_model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, None, 1)]         0         
                                                                 
 tcn (TCN)                   (None, None, 64)          136256    
                                                                 
 dense (Dense)               (None, None, 14)          910       
                                                                 
 activation (Activation)     (None, None, 14)          0         
                                                                 
Total params: 137,166
Trainable params: 137,166
Non-trainable params: 0
_________________________________________________________________


In [23]:
def get_intput_and_target(array_np, forecast_total: int = 14):
    """
    Gets input and target of model for a predictive task.
    :param array_np: Numpy array with shape (batch, features, 1).
    :returns:

      - :code:`x` same as numpy array without the last forecast_total samples
      - :code:`y` numpy array with shape (batch, features, forecast). Where last dimension corresponds to a different shift, starting from 1.
    """
    x = array_np[:, :-14]
    y = np.zeros(shape=x.shape[:-1] + (14,))

    for forecast_time in range(13):
        y[:,:,forecast_time] = array_np[:, (forecast_time+1):(-14+forecast_time+1)].squeeze()
    y[:,:,-1] = array_np[:, 14:].squeeze()
    return x, y

# The model, for sample i, needs to predict i+N (N being the shift y of last dimension)
x_train, y_train = get_intput_and_target(train_np)
x_val, y_val = get_intput_and_target(val_np)
x_train.shape

(5, 897, 1)

In [24]:
epochs=1000
history = tcn_model.fit(epochs=epochs, x=x_train, y=y_train, validation_data=(x_val, y_val))

Epoch 1/1000
1/1 [==============================] - 7s 7s/step - loss: 193.8504 - val_loss: 48.7200
Epoch 2/1000
1/1 [==============================] - 0s 225ms/step - loss: 57.3848 - val_loss: 34.5951
Epoch 3/1000
1/1 [==============================] - 0s 231ms/step - loss: 39.4289 - val_loss: 24.6042
Epoch 4/1000
1/1 [==============================] - 0s 206ms/step - loss: 24.5623 - val_loss: 18.0577
Epoch 5/1000
1/1 [==============================] - 0s 348ms/step - loss: 15.4551 - val_loss: 14.8229
Epoch 6/1000
1/1 [==============================] - 0s 349ms/step - loss: 11.9778 - val_loss: 13.3900
Epoch 7/1000
1/1 [==============================] - 0s 310ms/step - loss: 11.0160 - val_loss: 12.4961
Epoch 8/1000
1/1 [==============================] - 0s 339ms/step - loss: 10.5618 - val_loss: 11.5293
Epoch 9/1000
1/1 [==============================] - 0s 392ms/step - loss: 9.8744 - val_loss: 10.3438
Epoch 10/1000
1/1 [==============================] - 0s 369ms/step - loss: 8.8876 - v

In [25]:
train_history = history.history
train_history.update({'epochs': list(range(epochs))})
px.line(
    pd.DataFrame(
        train_history
    ),
    x='epochs',
    y=['loss', 'val_loss']
)

In [26]:
prediction = tcn_model.predict(all_data_np)
unnormalized_prediction = []
for i, scale in enumerate(scalers):
    unnormalized_prediction.append(scale.inverse_transform(prediction[i]))
unnormalized_prediction = np.ceil(np.array(unnormalized_prediction)).astype(int)  # Scale up because better to miss on more

1/1 [==============================] - 1s 568ms/step


In [27]:
unnormalized_prediction.shape

(5, 1093, 14)

In [28]:
tcn_model.predict(all_data_np).shape

1/1 [==============================] - 0s 77ms/step


(5, 1093, 14)

In [29]:
for forecast_time in range(1, 15):
  df_filled[f'prediction_{forecast_time}'] = np.nan
  for i, lounge in enumerate(df_filled['lounge_name'].unique()):
      mask = df_filled['lounge_name'] == lounge
      indices = df_filled.loc[mask].iloc[forecast_time:].index
      df_filled.loc[indices, f'prediction_{forecast_time}'] = unnormalized_prediction[i, :-forecast_time, forecast_time-1]
df_filled[df_filled['lounge_name'] == lounge].head(14)

,date,lounge_name,total_guests,day_of_week,month,year,previous_date_total_guests,previous_day_of_week_total_guests,previous_month_total_guests,prediction_1,...,prediction_5,prediction_6,prediction_7,prediction_8,prediction_9,prediction_10,prediction_11,prediction_12,prediction_13,prediction_14
4,2017-01-01,G,128,Sunday,1,2017,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,2017-01-02,G,205,Monday,1,2017,128.0,NaN,128.0,257.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10,2017-01-03,G,218,Tuesday,1,2017,205.0,NaN,205.0,279.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
19,2017-01-04,G,266,Wednesday,1,2017,218.0,NaN,218.0,299.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
20,2017-01-05,G,300,Thursday,1,2017,266.0,NaN,266.0,345.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
29,2017-01-06,G,204,Friday,1,2017,300.0,NaN,300.0,316.0,...,332.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
30,2017-01-07,G,200,Saturday,1,2017,204.0,NaN,204.0,295.0,...,299.0,413.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
39,2017-01-08,G,341,Sunday,1,2017,200.0,128.0,200.0,276.0,...,412.0,325.0,406.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
40,2017-01-09,G,537,Monday,1,2017,341.0,205.0,341.0,385.0,...,410.0,391.0,486.0,378.0,NaN,NaN,NaN,NaN,NaN,NaN
49,2017-01-10,G,484,Tuesday,1,2017,537.0,218.0,537.0,362.0,...,357.0,415.0,386.0,390.0,338.0,NaN,NaN,NaN,NaN,NaN


In [30]:
# import plotly.graph_objects as go

fig = px.line(
    df_filled,
    x='date',
    y=['total_guests', 'prediction_1', 'prediction_14'],
    facet_col='lounge_name',
    facet_col_wrap=3
)
fig.add_vline(x=split_date.timestamp() * 1000, annotation_text="test")
fig.show()

# Result Analysis

In [31]:
errors = []
for forecast_time in range(1, 15):
  df_filled[f'error_{forecast_time}'] = df_filled['total_guests'] - df_filled[f'prediction_{forecast_time}']
  errors.append(f'error_{forecast_time}')
df_filled['error_benchmark'] = df_filled['total_guests'] - df_filled['previous_day_of_week_total_guests']
errors.append('error_benchmark')

In [32]:
df_filled[errors].describe()
# Conclusions:
# - STD increases with forecast time (quite expected)
# - Bias to predict less than actually happens. This is bad, I would like actually the contrary.
#   - Can just add an offset, but it would be classy to change the loss function to penalize this.
# - 50% of the time I am under 100 people error.

,error_1,error_2,error_3,error_4,error_5,error_6,error_7,error_8,error_9,error_10,error_11,error_12,error_13,error_14,error_benchmark
count,5460.000000,5455.000000,5450.000000,5445.000000,5440.000000,5435.000000,5430.000000,5425.000000,5420.000000,5415.000000,5410.000000,5405.000000,5400.000000,5395.000000,5430.000000
mean,4.399084,3.922640,-0.482936,3.499174,-1.028493,-1.067525,1.868140,4.180276,7.659779,1.896768,6.322551,3.167068,5.256667,4.157183,0.952302
std,179.515927,194.044584,189.605767,203.378396,192.853767,191.459372,193.640937,199.129384,201.383498,197.240288,199.218961,201.082647,198.923016,206.359101,241.980437
min,-1664.000000,-1607.000000,-1497.000000,-1510.000000,-1398.000000,-1903.000000,-1591.000000,-1308.000000,-1603.000000,-1624.000000,-1557.000000,-1609.000000,-1403.000000,-1568.000000,-1724.000000
25%,-90.000000,-91.000000,-93.000000,-93.000000,-91.000000,-92.500000,-91.000000,-96.000000,-89.000000,-94.000000,-93.000000,-96.000000,-95.000000,-97.000000,-111.000000
50%,3.000000,6.000000,2.000000,3.000000,5.000000,8.000000,2.000000,6.000000,6.000000,6.000000,9.000000,8.000000,7.000000,8.000000,1.000000
75%,97.000000,102.000000,96.000000,109.000000,98.000000,99.000000,101.000000,106.000000,109.000000,99.000000,108.000000,105.000000,108.000000,108.000000,113.000000
max,1058.000000,1116.000000,976.000000,1201.000000,1121.000000,984.000000,1167.000000,1301.000000,1147.000000,1139.000000,1031.000000,1175.000000,1139.000000,1055.000000,1635.000000


In [33]:
df_filled[df_filled['lounge_name'] == 'G'][errors].describe()

,error_1,error_2,error_3,error_4,error_5,error_6,error_7,error_8,error_9,error_10,error_11,error_12,error_13,error_14,error_benchmark
count,1092.000000,1091.000000,1090.000000,1089.000000,1088.000000,1087.000000,1086.000000,1085.000000,1084.000000,1083.000000,1082.000000,1081.000000,1080.000000,1079.000000,1086.000000
mean,-3.142857,0.810266,-0.515596,-0.683196,1.633272,2.436063,-1.512891,-3.268203,-0.348708,0.846722,1.774492,2.500463,3.347222,-1.291010,-0.064457
std,79.302376,85.966111,85.906735,96.323519,89.616246,90.499999,88.232294,87.617201,87.979464,84.407025,87.436304,91.683868,89.971681,92.096025,121.612929
min,-394.000000,-373.000000,-428.000000,-481.000000,-500.000000,-409.000000,-424.000000,-458.000000,-420.000000,-385.000000,-411.000000,-379.000000,-484.000000,-408.000000,-517.000000
25%,-51.000000,-50.000000,-48.750000,-54.000000,-52.000000,-48.500000,-50.000000,-53.000000,-57.250000,-49.500000,-49.000000,-54.000000,-50.250000,-53.000000,-57.750000
50%,-2.000000,5.000000,4.000000,3.000000,5.000000,7.000000,1.500000,0.000000,2.500000,6.000000,5.500000,8.000000,7.500000,7.000000,2.000000
75%,47.250000,56.000000,52.000000,60.000000,59.000000,56.000000,54.000000,57.000000,56.000000,55.000000,58.000000,60.000000,64.000000,63.000000,66.000000
max,227.000000,335.000000,339.000000,328.000000,334.000000,371.000000,318.000000,262.000000,287.000000,355.000000,260.000000,330.000000,325.000000,219.000000,481.000000


In [34]:
px.box(df_filled, x='lounge_name', y=['error_1'],  notched=True, color='lounge_name', points="all")

In [35]:
fig = px.line(
    df_filled,
    x='date',
    y=['error_1', 'error_benchmark'],
    facet_col='lounge_name',
    facet_col_wrap=3
)
fig.add_vline(x=split_date.timestamp() * 1000, annotation_text="test")
fig.show()

In [36]:
px.histogram(df_filled, x='error_1', color='lounge_name', barmode='group')

# Create prediction and save

In [37]:
x, y = get_intput_and_target(all_data_np)                      # Retrain now with full dataset
history = tcn_model.fit(x=x, y=y, epochs=5000, verbose=1)      # Why not re-train on previous one?
prediction = tcn_model.predict(all_data_np)

Streaming output truncated to the last 5000 lines.
Epoch 2502/5000
1/1 [==============================] - 0s 362ms/step - loss: 0.1790
Epoch 2503/5000
1/1 [==============================] - 0s 317ms/step - loss: 0.1788
Epoch 2504/5000
1/1 [==============================] - 0s 362ms/step - loss: 0.1785
Epoch 2505/5000
1/1 [==============================] - 0s 264ms/step - loss: 0.1780
Epoch 2506/5000
1/1 [==============================] - 0s 212ms/step - loss: 0.1778
Epoch 2507/5000
1/1 [==============================] - 0s 209ms/step - loss: 0.1779
Epoch 2508/5000
1/1 [==============================] - 0s 214ms/step - loss: 0.1780
Epoch 2509/5000
1/1 [==============================] - 0s 227ms/step - loss: 0.1779
Epoch 2510/5000
1/1 [==============================] - 0s 235ms/step - loss: 0.1778
Epoch 2511/5000
1/1 [==============================] - 0s 223ms/step - loss: 0.1776
Epoch 2512/5000
1/1 [==============================] - 0s 213ms/step - loss: 0.1775
Epoch 2513/5000
1/1 [====

In [38]:
result_df = pd.DataFrame(columns=['date', 'lounge_name', 'total_guests'])
for i, lounge in enumerate(df_filled['lounge_name'].unique()):
    # import pdb; pdb.set_trace()
    df = pd.DataFrame({
        'date': pd.date_range(start="2019-12-30", end="2020-01-12"),
        'total_guests': np.ceil(scalers[i].inverse_transform(prediction[i])[-1, :]).astype(int),
        'lounge_name': [lounge,] * 14,
    })
    result_df = pd.concat([result_df, df])
result_df.sort_values(['date', 'lounge_name'], inplace=True, ignore_index=True)
result_df

<ipython-input-38-5d14ec98b8f8>:9: FutureWarning:

The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.



,date,lounge_name,total_guests
0,2019-12-30,EK,1738
1,2019-12-30,EL,1886
2,2019-12-30,EM,1020
3,2019-12-30,F,2026
4,2019-12-30,G,372
...,...,...,...
65,2020-01-12,EK,1738
66,2020-01-12,EL,1680
67,2020-01-12,EM,1047
68,2020-01-12,F,2162


In [39]:
result_df.to_csv('/content/lounge_attendance_predictions.csv', index=False)

In [40]:
df1 = pd.read_csv(Path('/content/lounge_attendance.csv'))
df1['type'] = 'data'
df2 = pd.read_csv(Path('/content/lounge_attendance_predictions.csv'))
df2['type'] = 'prediction'

px.line(
    pd.concat([df1, df2]),
    x='date',
    y='total_guests',
    color='type',
    facet_col='lounge_name',
    facet_col_wrap=3
)

# Conclusion

## Data Exploration

- Data had missing dates
- High seasonality and trend

## Model

Nothing was done for the fact that 'favor a modelization that would predict slightly more guests than there are'.
The only thing was a ceil for the final output. However, some ideas:

- Do a loss function that penalizes more to predict less than more.
- Fit a gauss function on the error curve and create a shifted threshold. Error theory will even let you choose a threshold that a assures with a certain confidence, that you won't have underpredictions.

## Bonus Question 1:

Error theory says that if EK, EL and EM has 1% error, the sum will also have % error. However, rounding errors (I do ceil) will introduce a higher error. i.e. if all EK, EL and EM have 100.3 guests, they will be predicted as 101, E would have 303 with this method but a more accurate prediction would be 301.

## Bonus Question 2:

I will probably use a similar approach as the gauss graph I did before. Take into account to use it only on dates not used for training of course.
It would be better to use percentage instead of guests error, as they are supposed to increase per year if the yearly tendency is indeed to increase.
